## Régression régularisée 
---

Pour cette partie on utilise le même jeu de données utilisé dans le cas de la régression linéaire.

In [3]:
from sklearn.datasets import make_regression
import numpy as np

In [4]:
np.random.seed(0)

x, y = make_regression(n_samples=100, n_features=1, noise=10)
biais = np.ones((x.shape))

y = y.reshape((100,1))
X = np.hstack((x, biais))

### Le sur-apprentissage ou Overfiting 

L'overfiting consiste pour un modèle de bien apprendre sur les données d'apprentissage mais ne pas réussir à bien prédire à l'arrivée de nouvelles données. Ce modèle aura capturer trop de complexités (bruit aléatoire) de l'échantillon d'entraînement et ne sera pas en mesure de généraliser sur de nouvelles données. Plusieurs facteurs engendre l'overfitting : 

- **La complexité du modèle** : lorsque qu'un modèle possède beaucoup de coefficients, ce dernier a beaucoup plus de chance d'expliquer trop de détails liés au bruit de variabilité. Ceci fait que notre population à un moment données, présente une corrélation hasardeuse qui ne fonctionnerait pas sur un autre échantillon. Au niveau statistique le $t$ Student donnerait un p-value inférieur à 5% alors que cette corrélation n'est que le fruit du hasard. Ceci s'explique par le fait que la distribution des *p-values* est uniformes. Donc à partir de 20 variables il y a une variable corrélée significativement (seuil de 5%) avec notre variable de sortie.
- **La multicolinéarité** : certaines variables d’entrés peuvent être liées ce qui engendre de la redondance d'informations et peut conduire à modifier les estimateurs.
- **La haute dimensionnalité** : si un modèle possède trop de coefficients, comparé à son nombre d'individus, il aura tendance à apprendre beaucoup sur ce petit échantillon d'individus.


### Régression Ridge : pénalisation $L_2$

La régression **Ridge** ajoute un paramètre de pénalité (égale à la distance euclidienne des paramètres) afin de forcer les coefficients à être le plus petit possible, sans les réduire à zéros :

$$Ridge = \sum(y_i - X_i \beta)^2 + \lambda \vert\vert \, \beta \, \vert\vert_2$$

Pour le calculer :

$$\beta = (X^TX + \lambda I)^{-1}X^Ty$$

In [5]:
def ridge(X, y, penalty):
    return np.linalg.inv(
        X.T.dot(X) + penalty * np.identity(X.shape[1])
    ).dot(X.T.dot(y))

In [6]:
ridge(X, y, penalty=0.1)

array([[42.57746844],
       [-0.81086131]])

In [46]:
def loss(X, y, beta):
    n = len(y)
    return 1/(2*n) * np.sum((prediction(X, beta) - y)**2)

def gradient(X, y, beta):
    n = len(y)
    return 1/n * X.T.dot(prediction(X, beta) - y)

def prediction(X, beta):
    return X.dot(beta)

def ridge_gradient_descent(X, y, beta, learning_rate, n_iterations, penalty=0.0):

    cost_history = []
    n = len(y)
    for i in range(0, n_iterations):

        beta = beta - learning_rate * \
            (gradient(X, y, beta) + (penalty / n * beta))
        cost_history.append(loss(X, y, beta))

    return beta, cost_history

In [47]:
beta = np.random.randn(2, 1)

coef, loss_ridge = ridge_gradient_descent(X, y, beta, learning_rate=0.01,
                                          n_iterations=1000, penalty=0.1)

In [48]:
coef

array([[42.57565418],
       [-0.80981402]])

On peut vérifier avec l'API sklearn :

In [55]:
from sklearn.linear_model import Ridge

r = Ridge(alpha=0.1)
r.fit(X, y)

print(f"{r.coef_[0][0]} \n {r.intercept_[0]}")

42.577516134630706 
 -0.811675024493526


### Régression Lasso : pénalisation $L_1$

La régression Lasso pénalise les coefficients par la norme l1 (distance de manhattan). Cette contrainte a pour conséquence d'introduire du biais. Par cette régularisation, les coefficients les moins significatifs sont réduits vers zéro.

In [56]:
def lasso_gradient_descent(X, y, beta, learning_rate, n_iterations, penalty=0.0):
    cost_history = []
    n = len(y)
    
    for i in range(0, n_iterations):
        beta = beta - learning_rate * \
            (gradient(X, y, beta) + (penalty / n * sum(abs(beta))))
        
        cost_history.append(loss(X, y, beta))

    return beta, cost_history

In [73]:
lasso_coef, lasso_loss = lasso_gradient_descent(
    X, y, beta, learning_rate=0.01, n_iterations=1000, penalty=10)

In [74]:
lasso_coef

array([[38.59169807],
       [-4.92189438]])

In [ ]:
def lasso_eq(X, y, penalty):
    return np.linalg.inv(
        X.T.dot(X) + penalty * np.identity(X.shape[1])).dot(X.T.dot(y))

In [76]:
from sklearn.linear_model import Lasso

lass = Lasso(alpha=10).fit(X, y)
lass.coef_, lass.intercept_

(array([32.77523069,  0.        ]), array([-0.22541978]))

**Attention** : Toutes les hypothèses de test concernant la régression s’écroulent si on ajoute ce terme de pénalités. 

Cette régularisation s'utilise dans le cas où on veut juste bien prédire (sans contraintes). Si jamais il y avait un problème de **multicolinéarité**, la pénalisation peut régler ce problème d’overfitting.

**Rasoir d’Ockham** : parmi les solutions on veut la plus simple. 

In [81]:
l = LassoRegression(learning_rate=0.1, iterations=1000, l1_penality=0.1)

In [92]:
import pandas as pd

k = pd.DataFrame(X[:,0].reshape(len(X), 1))
m = pd.DataFrame(y)